<a href="https://colab.research.google.com/github/awhitehouse1/DS4002-Project-2/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np

In [21]:
# Pre-processing the data
df = pd.read_csv('/content/Final_time_series_data.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108336 entries, 0 to 108335
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Year                    108336 non-null  int64  
 1   Month                   108336 non-null  int64  
 2   new_deaths_per_million  108335 non-null  float64
 3   Country                 108336 non-null  object 
 4   Date                    108336 non-null  object 
 5   GDP_per_Capita          107217 non-null  float64
 6   Democracy score         108336 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 5.8+ MB
None


In [22]:
import numpy as np

# Replace Inf and -Inf values with NaN
data = data.replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN values
data_cleaned = data.dropna()

# Check that there are no remaining NaN or Inf values
print(data_cleaned.isna().sum())  # Should print 0 for all columns
print(np.isinf(data_cleaned).sum())  # Should print 0 for all columns

new_deaths_per_million    0
GDP_per_Capita            0
Democracy score           0
dtype: int64
new_deaths_per_million    0
GDP_per_Capita            0
Democracy score           0
dtype: int64


In [23]:
# Set the date column as the index for time series analysis
df.set_index('Date', inplace=True)

# Inspect the final dataframe
print(df.head())

           Year  Month  new_deaths_per_million      Country  GDP_per_Capita  \
Date                                                                          
1/22/2020  2020      1                     0.0  Afghanistan      512.055098   
1/23/2020  2020      1                     0.0  Afghanistan      512.055098   
1/24/2020  2020      1                     0.0  Afghanistan      512.055098   
1/25/2020  2020      1                     0.0  Afghanistan      512.055098   
1/26/2020  2020      1                     0.0  Afghanistan      512.055098   

           Democracy score  
Date                        
1/22/2020             2.85  
1/23/2020             2.85  
1/24/2020             2.85  
1/25/2020             2.85  
1/26/2020             2.85  


In [ ]:
# Vector AutoRegression (VAR) Model

In [24]:
!pip install statsmodels

In [25]:
from statsmodels.tsa.api import VAR

# Fit the VAR model with the cleaned data
model = VAR(data_cleaned)
results = model.fit(maxlags=15, ic='aic')

# Print model summary
print(results.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 22, Oct, 2024
Time:                     01:44:43
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    11.9446
Nobs:                     107202.    HQIC:                   11.9365
Log likelihood:      -1.09583e+06    FPE:                    152213.
AIC:                      11.9330    Det(Omega_mle):         152030.
--------------------------------------------------------------------
Results for equation new_deaths_per_million
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                             -0.037929         0.028978           -1.309           0.191
L1.new_deaths_per_million         -0.001336         0.003040           -0.440        

In [ ]:
# Correlation matrix of residuals shows the correlations between the residuals
# (errors) of the different variables
# new_deaths_per_million and GDP_per_Capita: 0.015304
# Very close to zero which suggests that the errors of these two variables are
# almost uncorrelated, meaning there is no strong linear relationship between the
# prediction errors of these variabls

# new_deaths_per_million and democracy score: 0.028133
# Also close to zero which suggests a weak or non-existent correlation between
# the errors of these two variables

# GDP_per_Capita and democracy score: 0.481471
# Moderate positive correlation which indicates that when the model makes errors
# in predicting GDP_per_Capita, those errors are moderately correlated with errors
# in predicting democracy score

# You want the residuals of different variables to be uncorrelated (close to 0)
# which indicates that the model is doing a good job of capturing the relationships
# between the variables, and the remaining prediction errors are random

# GDP_per_capita and democracy score suggest that the model might not be fully
# capturing the relationship between these variables

In [26]:
# Granger Casuality Test
from statsmodels.tsa.stattools import grangercausalitytests

# Specify the maxlag (based on your VAR model)
max_lag = 15

# Test Granger Causality for GDP per Capita on new_deaths_per_million
print("Granger Causality Test: GDP per Capita -> new_deaths_per_million")
grangercausalitytests(data_cleaned[['new_deaths_per_million', 'GDP_per_Capita']], max_lag)

# Test Granger Causality for Democracy score on new_deaths_per_million
print("\nGranger Causality Test: Democracy score -> new_deaths_per_million")
grangercausalitytests(data_cleaned[['new_deaths_per_million', 'Democracy score']], max_lag)

Granger Causality Test: GDP per Capita -> new_deaths_per_million

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=58.9570 , p=0.0000  , df_denom=107212, df_num=1
ssr based chi2 test:   chi2=58.9587 , p=0.0000  , df=1
likelihood ratio test: chi2=58.9425 , p=0.0000  , df=1
parameter F test:         F=58.9570 , p=0.0000  , df_denom=107212, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=31.2097 , p=0.0000  , df_denom=107209, df_num=2
ssr based chi2 test:   chi2=62.4223 , p=0.0000  , df=2
likelihood ratio test: chi2=62.4041 , p=0.0000  , df=2
parameter F test:         F=31.2097 , p=0.0000  , df_denom=107209, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=23.9481 , p=0.0000  , df_denom=107206, df_num=3
ssr based chi2 test:   chi2=71.8489 , p=0.0000  , df=3
likelihood ratio test: chi2=71.8249 , p=0.0000  , df=3
parameter F test:         F=23.9481 , p=0.0000  , df_denom=107206, df_num=3

Granger Ca

{1: ({'ssr_ftest': (606.6308396088929, 1.4257742912945901e-133, 107212.0, 1),
   'ssr_chi2test': (606.6478143180562, 5.995973000586673e-134, 1),
   'lrtest': (604.9379826710792, 1.4117434389264817e-133, 1),
   'params_ftest': (606.6308396083098,
    1.4257742917049255e-133,
    107212.0,
    1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (322.78845890900294, 1.7184667937477025e-140, 107209.0, 2),
   'ssr_chi2test': (645.607026153958, 6.430067495626328e-141, 2),
   'lrtest': (643.6709790957393, 1.6928668815044586e-140, 2),
   'params_ftest': (322.78845890882127,
    1.7184667940560893e-140,
    107209.0,
    2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (229.92385294139916, 1.04312861881758e-148, 107206.0, 3),
   'ssr_chi2test': (689.8165973566654, 3.389723068418739e-149, 3),
   'lrtest': (687.6069042888703, 1.0216409236527534e-148, 3),
   'params_ftest': (229.9238529428841,
    1.0431286165091664e-148,
    107206.0,
    3.0)},
   a

In [ ]:
# GDP per capita - New Deaths per million
# At 1-6 lags, the p-value is 0.0000, meaning that GDP per capita granger causes
# new deaths per million with strong statistical significance
# F-test and chi-squared values confirm that GDP per capita can help predict changes
# in new deaths per million during these time periods
# From 7+, the p-value becomes very high, indicating GDP per capita no longer
# granger causes new deaths per million

# Democracy score - New deaths per million
# At all lags 1-5, the p-values are 0.000, meaning democracy score strongly grander
# causes new deaths per million
# High F-test and chi-squared values indicate strong predictive power of democracy
# score on new deaths per million

In [27]:
# SARIMAX Model
pip install statsmodels pandas matplotlib

In [28]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Select the target and exogenous variables
y = df['new_deaths_per_million']
exog = df[['GDP_per_Capita', 'Democracy score']]

# Define the SARIMAX model (replace order and seasonal_order with appropriate values)
model = SARIMAX(y, exog=exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# Fit the model
sarimax_model = model.fit(disp=False)

# Print model summary
print(sarimax_model.summary())


MissingDataError: exog contains inf or nans